## Importing Required Libraries

In [ ]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import numpy as np
import time
import faiss
import re

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from google.colab import files
uploaded = files.upload()

df = pd.read_csv("mock_sales_cleaned.csv")
df["dt"] = pd.to_datetime(df["dt"], errors="coerce").dt.strftime("%Y-%m-%d")

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer.pad_token = tokenizer.eos_token

def generate_sentence(row):
    try:
        row_text = (
            f"On {row['dt']}, product {row['productId']} had {row['openCardCount']} product card views, "
            f"{row['addToCartCount']} items added to cart, resulting in {row['ordersCount']} orders worth "
            f"{row['ordersSumRub']} rubles. {row['buyoutsCount']} buyouts occurred, worth {row['buyoutsSumRub']} rubles, "
            f"with a buyout percentage of {row['buyoutPercent']}%. "
            f"Add-to-cart conversion: {row['addToCartConversion']}%, cart-to-order conversion: {row['cartToOrderConversion']}%."
        )

        inputs = tokenizer(row_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=100,
            num_beams=3,  # reduce for speed
            temperature=0.7,
            no_repeat_ngram_size=2,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip().split('\n')[0]
    except Exception as e:
        print(f"Error on row {row['dt']} {row.get('productId', '')}: {e}")
        return ""

tqdm.pandas()
df["generated_sentence"] = df.progress_apply(generate_sentence, axis=1)

os.makedirs("generate_text", exist_ok=True)
output_path = "generate_text/wildberries_colab.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for sentence in df["generated_sentence"]:
        f.write(sentence + "\n")

files.download(output_path)


Using device: cuda


Saving mock_sales_cleaned.csv to mock_sales_cleaned (1).csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 50/50 [00:21<00:00,  2.34it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>